In [3]:
import pandas as pd

dataframe = pd.read_csv('dataset.csv')
dataframe.head()

,Unnamed: 0,Task,Importance,Time to complete (minutes)
0,0,Prepare presentation for client meeting,10,90
1,1,Pay medical bills,6,30
2,2,Submit tax forms,5,30
3,3,Schedule dentist appointment,6,30
4,4,Register for training session,7,45


In [4]:
raw_tasks = dataframe["Task"]
importances = dataframe["Importance"]
print(len(raw_tasks))

10000


In [5]:
tokens = [[token for token in task.lower().split()] for task in raw_tasks]
print(tokens[:5])

[['prepare', 'presentation', 'for', 'client', 'meeting'], ['pay', 'medical', 'bills'], ['submit', 'tax', 'forms'], ['schedule', 'dentist', 'appointment'], ['register', 'for', 'training', 'session']]


In [6]:
vocabulary = set()
for task in tokens:
    for token in task:
        vocabulary.add(token)
vocabulary = list(vocabulary)
print(len(vocabulary))

1698


In [7]:
import numpy as np

word2index = dict()
for i, w in enumerate(vocabulary):
    word2index[w] = i
indices = [np.asarray([word2index[token] for token in task]) for task in tokens]
print(indices[:5])

[array([ 468,  374,  773,  146, 1478]), array([ 869,  618, 1248]), array([ 847,  656, 1240]), array([992, 967, 554]), array([913, 773, 949, 831])]


In [6]:
from autograd import Tensor
from layers import Embedding, LSTMCell, MSELoss, Linear
from optimisers import SGD

embedding = Embedding(vocab_size=len(vocabulary), dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_outputs=1)
output_layer = Linear(n_inputs=512, n_outputs=1)

In [7]:
criterion = MSELoss()
optimiser = SGD(parameters=model.get_parameters() + embedding.get_parameters() + output_layer.get_parameters(), alpha=0.05)

In [61]:
X_train = indices[:int(len(indices) * 0.8)]
y_train = importances[:int(len(indices) * 0.8)]

X_test = indices[int(len(indices) * 0.8):]
y_test = importances[int(len(indices) * 0.8):]

print(type(y_train))

<class 'pandas.core.series.Series'>


In [9]:
min_loss = 1000
iterations = 10
for iteration in range(iterations):
    total_loss = 0
    
    hidden = model.init_hidden(batch_size=1)
    
    for task_i in range(len(X_train)):
        hidden = model.init_hidden(batch_size=1)
        
        for t in range(len(X_train[task_i])):
            input = Tensor([X_train[task_i][t]], autograd=True)
            rnn_input = embedding.forward(input=input)
            hidden = model.forward(input=rnn_input, hidden=hidden)
        
        output = output_layer.forward(hidden[0])
        target = Tensor(y_train[task_i], autograd=True)
        loss = criterion.forward(output, target)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.data / len(X_train[task_i])
        epoch_loss = np.exp(total_loss / (task_i + 1))
        
        if epoch_loss < min_loss:
            min_loss = epoch_loss
            print()
        
        print(f"Iter: {iteration} - Alpha: {str(optimiser.alpha)[:5]} - Example {task_i + 1}/{len(X_train)} - Min Loss: {str(min_loss)[:5]} - Loss: {epoch_loss}", end='\r')
    optimiser.alpha *= 0.99

Iter: 0 - Alpha: 0.05 - Example 4/8000 - Min Loss: 1000 - Loss: [2185.6413197]1]]
Iter: 0 - Alpha: 0.05 - Example 5/8000 - Min Loss: [619. - Loss: [619.50600201]
Iter: 0 - Alpha: 0.05 - Example 6/8000 - Min Loss: [236. - Loss: [236.20143532]
Iter: 0 - Alpha: 0.05 - Example 7/8000 - Min Loss: [109. - Loss: [109.50493138]
Iter: 0 - Alpha: 0.05 - Example 8/8000 - Min Loss: [60.9 - Loss: [60.93181182]
Iter: 0 - Alpha: 0.05 - Example 14/8000 - Min Loss: [56.0 - Loss: [66.48309715]]
Iter: 0 - Alpha: 0.05 - Example 15/8000 - Min Loss: [51.0 - Loss: [51.04762177]
Iter: 0 - Alpha: 0.05 - Example 16/8000 - Min Loss: [42.1 - Loss: [42.18814785]
Iter: 0 - Alpha: 0.05 - Example 17/8000 - Min Loss: [34.2 - Loss: [34.24174177]
Iter: 0 - Alpha: 0.05 - Example 18/8000 - Min Loss: [29.6 - Loss: [29.69959271]
Iter: 0 - Alpha: 0.05 - Example 19/8000 - Min Loss: [24.8 - Loss: [24.86774854]
Iter: 0 - Alpha: 0.05 - Example 20/8000 - Min Loss: [21.1 - Loss: [21.1789909]
Iter: 0 - Alpha: 0.05 - Example 21/8000

Iter: 0 - Alpha: 0.05 - Example 549/8000 - Min Loss: [3.28 - Loss: [3.28996776]
Iter: 0 - Alpha: 0.05 - Example 566/8000 - Min Loss: [3.28 - Loss: [3.28646226]
Iter: 0 - Alpha: 0.05 - Example 567/8000 - Min Loss: [3.28 - Loss: [3.28116366]
Iter: 0 - Alpha: 0.05 - Example 568/8000 - Min Loss: [3.28 - Loss: [3.28010162]
Iter: 0 - Alpha: 0.05 - Example 570/8000 - Min Loss: [3.27 - Loss: [3.28108807]
Iter: 0 - Alpha: 0.05 - Example 574/8000 - Min Loss: [3.27 - Loss: [3.27949252]
Iter: 0 - Alpha: 0.05 - Example 575/8000 - Min Loss: [3.27 - Loss: [3.27287042]
Iter: 0 - Alpha: 0.05 - Example 576/8000 - Min Loss: [3.26 - Loss: [3.26621118]
Iter: 0 - Alpha: 0.05 - Example 577/8000 - Min Loss: [3.26 - Loss: [3.26247576]
Iter: 0 - Alpha: 0.05 - Example 578/8000 - Min Loss: [3.25 - Loss: [3.25659041]
Iter: 0 - Alpha: 0.05 - Example 582/8000 - Min Loss: [3.25 - Loss: [3.25239475]
Iter: 0 - Alpha: 0.05 - Example 583/8000 - Min Loss: [3.24 - Loss: [3.24797214]
Iter: 0 - Alpha: 0.05 - Example 584/8000

Iter: 0 - Alpha: 0.05 - Example 1463/8000 - Min Loss: [2.84 - Loss: [2.85111644]
Iter: 0 - Alpha: 0.05 - Example 1475/8000 - Min Loss: [2.84 - Loss: [2.85069707]
Iter: 0 - Alpha: 0.05 - Example 1476/8000 - Min Loss: [2.84 - Loss: [2.84887984]
Iter: 0 - Alpha: 0.05 - Example 1477/8000 - Min Loss: [2.84 - Loss: [2.84691123]
Iter: 0 - Alpha: 0.05 - Example 1478/8000 - Min Loss: [2.84 - Loss: [2.84557434]
Iter: 0 - Alpha: 0.05 - Example 1485/8000 - Min Loss: [2.84 - Loss: [2.84535438]
Iter: 0 - Alpha: 0.05 - Example 1486/8000 - Min Loss: [2.84 - Loss: [2.84335318]
Iter: 0 - Alpha: 0.05 - Example 1497/8000 - Min Loss: [2.84 - Loss: [2.84178319]
Iter: 0 - Alpha: 0.05 - Example 1498/8000 - Min Loss: [2.83 - Loss: [2.83980271]
Iter: 0 - Alpha: 0.05 - Example 1499/8000 - Min Loss: [2.83 - Loss: [2.83783006]
Iter: 0 - Alpha: 0.05 - Example 1533/8000 - Min Loss: [2.83 - Loss: [2.83744177]
Iter: 0 - Alpha: 0.05 - Example 1534/8000 - Min Loss: [2.83 - Loss: [2.83556487]
Iter: 0 - Alpha: 0.05 - Exam

Iter: 0 - Alpha: 0.05 - Example 3897/8000 - Min Loss: [2.72 - Loss: [2.72267364]
Iter: 0 - Alpha: 0.05 - Example 3899/8000 - Min Loss: [2.72 - Loss: [2.72233329]
Iter: 0 - Alpha: 0.05 - Example 3900/8000 - Min Loss: [2.72 - Loss: [2.72180649]
Iter: 0 - Alpha: 0.05 - Example 3901/8000 - Min Loss: [2.72 - Loss: [2.72121077]
Iter: 0 - Alpha: 0.05 - Example 3902/8000 - Min Loss: [2.72 - Loss: [2.72109326]
Iter: 0 - Alpha: 0.05 - Example 3903/8000 - Min Loss: [2.72 - Loss: [2.72046223]
Iter: 0 - Alpha: 0.05 - Example 3904/8000 - Min Loss: [2.72 - Loss: [2.72027293]
Iter: 0 - Alpha: 0.05 - Example 3905/8000 - Min Loss: [2.71 - Loss: [2.71975862]
Iter: 0 - Alpha: 0.05 - Example 3906/8000 - Min Loss: [2.71 - Loss: [2.71914672]
Iter: 0 - Alpha: 0.05 - Example 3909/8000 - Min Loss: [2.71 - Loss: [2.71895308]
Iter: 0 - Alpha: 0.05 - Example 3910/8000 - Min Loss: [2.71 - Loss: [2.71829322]
Iter: 0 - Alpha: 0.05 - Example 3911/8000 - Min Loss: [2.71 - Loss: [2.71760853]
Iter: 0 - Alpha: 0.05 - Exam

Iter: 0 - Alpha: 0.05 - Example 4950/8000 - Min Loss: [2.64 - Loss: [2.6432193]]
Iter: 0 - Alpha: 0.05 - Example 4951/8000 - Min Loss: [2.64 - Loss: [2.64280772]
Iter: 0 - Alpha: 0.05 - Example 5003/8000 - Min Loss: [2.64 - Loss: [2.64277898]
Iter: 0 - Alpha: 0.05 - Example 5004/8000 - Min Loss: [2.64 - Loss: [2.64250768]
Iter: 0 - Alpha: 0.05 - Example 5005/8000 - Min Loss: [2.64 - Loss: [2.64206744]
Iter: 0 - Alpha: 0.05 - Example 5006/8000 - Min Loss: [2.64 - Loss: [2.64160358]
Iter: 0 - Alpha: 0.05 - Example 5007/8000 - Min Loss: [2.64 - Loss: [2.64126039]
Iter: 0 - Alpha: 0.05 - Example 5008/8000 - Min Loss: [2.64 - Loss: [2.64120657]
Iter: 0 - Alpha: 0.05 - Example 5009/8000 - Min Loss: [2.64 - Loss: [2.6408765]
Iter: 0 - Alpha: 0.05 - Example 5016/8000 - Min Loss: [2.64 - Loss: [2.64086321]
Iter: 0 - Alpha: 0.05 - Example 5017/8000 - Min Loss: [2.64 - Loss: [2.6403531]
Iter: 0 - Alpha: 0.05 - Example 5019/8000 - Min Loss: [2.64 - Loss: [2.64041153]
Iter: 0 - Alpha: 0.05 - Exampl

Iter: 0 - Alpha: 0.05 - Example 7696/8000 - Min Loss: [2.59 - Loss: [2.59080944]
Iter: 0 - Alpha: 0.05 - Example 7706/8000 - Min Loss: [2.59 - Loss: [2.59079672]
Iter: 0 - Alpha: 0.05 - Example 7707/8000 - Min Loss: [2.59 - Loss: [2.59047673]
Iter: 0 - Alpha: 0.05 - Example 7745/8000 - Min Loss: [2.59 - Loss: [2.59051556]
Iter: 0 - Alpha: 0.05 - Example 7746/8000 - Min Loss: [2.59 - Loss: [2.59026318]
Iter: 0 - Alpha: 0.05 - Example 7747/8000 - Min Loss: [2.59 - Loss: [2.59008117]
Iter: 0 - Alpha: 0.05 - Example 7748/8000 - Min Loss: [2.58 - Loss: [2.589794]
Iter: 0 - Alpha: 0.05 - Example 7749/8000 - Min Loss: [2.58 - Loss: [2.58947661]
Iter: 0 - Alpha: 0.05 - Example 7752/8000 - Min Loss: [2.58 - Loss: [2.58924038]
Iter: 0 - Alpha: 0.05 - Example 7753/8000 - Min Loss: [2.58 - Loss: [2.58900352]
Iter: 0 - Alpha: 0.05 - Example 7754/8000 - Min Loss: [2.58 - Loss: [2.58869659]
Iter: 0 - Alpha: 0.05 - Example 7755/8000 - Min Loss: [2.58 - Loss: [2.58844102]
Iter: 0 - Alpha: 0.05 - Exampl

KeyboardInterrupt: 

In [48]:
def predict(task_indices):
    hidden = model.init_hidden(batch_size=1)
    for t in range(len(task_indices)):
        input = Tensor([task_indices[t]], autograd=True)
        rnn_input = embedding.forward(input=input)
        hidden = model.forward(input=rnn_input, hidden=hidden)
    output = output_layer.forward(hidden[0])
    
    return output

In [70]:
for task_i in range(len(X_test)):
    output = predict(X_test[task_i])
    target = Tensor(np.array(y_test)[task_i], autograd=True)
    loss = criterion.forward(output, target)
    print(f"Task {task_i + 1}/{len(X_test)} - Loss: {loss.data}", end='\r')
    total_loss += loss.data / len(X_test[task_i])
loss_to_display = np.exp(total_loss / (task_i + 1))
print()
print(loss_to_display)

Task 2000/2000 - Loss: [0.13342608]]]5]
[33.65812669]


In [8]:
import pickle as pkl

with open("importance_embedding.pkl", "wb") as file:
    pkl.dump(embedding, file)

with open("importance_model.pkl", "wb") as file:
    pkl.dump(model, file)

with open("importance_output.pkl", "wb") as file:
    pkl.dump(output_layer, file)

with open("word2index.pkl", "wb") as file:
    pkl.dump(word2index, file)

NameError: name 'embedding' is not defined